In [ ]:
print('Loading class: PioneerP3DX_base')
class PioneerP3DX_base:

    def __init__(self, api: VRep):
        self._api = api
        self._left_motor = api.joint.with_velocity_control("Pioneer_p3dx_leftMotor")
        self._right_motor = api.joint.with_velocity_control("Pioneer_p3dx_rightMotor")
        self._sensor_rgb_left = api.sensor.vision("LeftRGBSensor")  # type: VisionSensor
        self._sensor_rgb_right = api.sensor.vision("RightRGBSensor")  # type: VisionSensor
        self._sensor_ultrasonic_left = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor3")
        self._sensor_ultrasonic_right = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor6")
        
print('This is a base class for the {} model\n'.format(PioneerP3DX_base.__name__ ))

In [ ]:
print('Loading class: PioneerP3DX')
class PioneerP3DX:

    def __init__(self, api: VRep):
        self._api = api
        self._left_motor = api.joint.with_velocity_control("Pioneer_p3dx_leftMotor")
        self._right_motor = api.joint.with_velocity_control("Pioneer_p3dx_rightMotor")
        self._sensor_ultrasonic_left = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor3")
        self._sensor_ultrasonic_right = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor6")

    
    def set_two_motor(self, left: float, right: float):
        self._left_motor.set_target_velocity(left)
        self._right_motor.set_target_velocity(right)

    def rotate_right(self, speed=2.0):
        self.set_two_motor(speed, -speed)

    def rotate_left(self, speed=2.0):
        self.set_two_motor(-speed, speed)

    def move_forward(self, speed=2.0):
        self.set_two_motor(speed, speed)

    def move_backward(self, speed=2.0):
        self.set_two_motor(-speed, -speed)
    
    def ultrasonic_right_length(self):
        return self._sensor_ultrasonic_right.read()[1].distance()

    def ultrasonic_left_length(self):
        return self._sensor_ultrasonic_left.read()[1].distance()

methods = [method for method in dir(PioneerP3DX) if not method.startswith('_')]
print('Methods available in {}:\n\t{}\n'.format(PioneerP3DX.__name__ , '\n\t'.join(methods)))

In [ ]:
print('Loading class: PioneerP3DXL')

#https://github.com/Troxid/vrep-api-python/blob/master/examples/line_follower_pioneer.py
class PioneerP3DXL:

    def __init__(self, api: VRep):
        self._api = api
        self._left_motor = api.joint.with_velocity_control("Pioneer_p3dx_leftMotor")
        self._right_motor = api.joint.with_velocity_control("Pioneer_p3dx_rightMotor")
        self._sensor_rgb_left = api.sensor.vision("LeftRGBSensor")  # type: VisionSensor
        self._sensor_rgb_right = api.sensor.vision("RightRGBSensor")  # type: VisionSensor
        self._sensor_ultrasonic_left = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor3")
        self._sensor_ultrasonic_right = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor6")

    
    def set_two_motor(self, left: float, right: float):
        self._left_motor.set_target_velocity(left)
        self._right_motor.set_target_velocity(right)

    def rotate_right(self, speed=2.0):
        self.set_two_motor(speed, -speed)

    def rotate_left(self, speed=2.0):
        self.set_two_motor(-speed, speed)

    def move_forward(self, speed=2.0):
        self.set_two_motor(speed, speed)

    def move_backward(self, speed=2.0):
        self.set_two_motor(-speed, -speed)

    def color_right(self) -> int:
        image = self._sensor_rgb_right.raw_image(is_grey_scale=True)  # type: List[int]
        average = sum(image) / len(image)
        return average

    def color_left(self) -> int:
        image = self._sensor_rgb_left.raw_image(is_grey_scale=True)  # type: List[int]
        average = sum(image) / len(image)
        return average
    
    def ultrasonic_right_length(self):
        return self._sensor_ultrasonic_right.read()[1].distance()

    def ultrasonic_left_length(self):
        return self._sensor_ultrasonic_left.read()[1].distance()


methods = [method for method in dir(PioneerP3DXL) if not method.startswith('_')]
print('Methods available in {}:\n\t{}\n'.format(PioneerP3DXL.__name__ , '\n\t'.join(methods)))

## Widget Displays

In [ ]:
from IPython.display import display
import ipywidgets as widgets

sensorText = widgets.Text(value='Left light sensor value', 
                          disabled=True, 
                          description='Left light:')

sensorTextMax = widgets.Text(value='Max left light sensor value', 
                             disabled=True, 
                             description='Max left light:')

print("The following text widgets are available for display: sensorText, sensorTextMax")